### Prepare space in /flash

In [1]:
import os, subprocess

In [2]:
cwd = os.path.basename(os.getcwd())
tmpdir = os.path.join("/flash/ReiterU/tmp/", cwd, "3D")
if not os.path.exists(tmpdir):
    os.makedirs(tmpdir)

### Main

In [3]:
import pandas as pd
import numpy as np
import os, h5py
from probemapper.plot import merge_data_oct
from probemapper.plot import threeD

QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/scratch/runtime-tomoyuki-mano1'


In [4]:
lut = pd.read_csv("/bucket/ReiterU/DBS/atlas/O_Laqueus_v0.2/region_LUT.csv")
lut["region_name2"] = ["VL", "Others", "sFL", "iFL", "Buc", "Subfr", "aBL", "dBL", "Subv", "Prec"]
display(lut)

,ID,region_name,region_color,region_name2
0,1,Vertical Lobe,"128,174,128",VL
1,3,Others,"216,100,79",Others
2,4,Median superior frontal lobe + Lateral superio...,"216,100,79",sFL
3,5,Median inferior frontal lobe + Lateral inferio...,"255,147,114",iFL
4,6,Superior buccal lobe\n+ Posterior buccal lobe,"244,128,26",Buc
5,7,Subfrontal lobe,"251,202,0",Subfr
6,8,Anterior basal lobe,"89,114,255",aBL
7,9,Anterior dorsal basal lobe + Posterior dorsal ...,"82,145,163",dBL
8,11,Subvertical Lobe,"24,151,54",Subv
9,12,Precomissural,"255,170,255",Prec


In [5]:
data = [
    {
        "name": "L-J30",
        "channel_lut": "/bucket/ReiterU/DBS/data/20220526_P189_laqueus_DiI_L-J30/analysis/indexing/channel_LUT.csv",
        "offset": 0,
        "hdf5_column": 0,
    },
    {
        "name": "L-F11",
        "channel_lut": "/bucket/ReiterU/DBS/data/20220526_P190_laqueus_DiI_L-F11/analysis/indexing/channel_LUT.csv",
        "offset": 0,
        "hdf5_column": 1,
    },
    {
        "name": "L-M4",
        "channel_lut": "/bucket/ReiterU/DBS/data/20220321_P177_laqueus_DiI_3-8_CUBIC/analysis/indexing/channel_LUT.csv",
        "offset": 0,
        "hdf5_column": 2,
    },
    {
        "name": "L-M13",
        "channel_lut": "/bucket/ReiterU/DBS/data/20220529_P191_laqueus_DiI_LM13/analysis/indexing/channel_LUT.csv",
        "offset": 10,
        "hdf5_column": 3,
    },
    {
        "name": "L-M27",
        "channel_lut": "/bucket/ReiterU/DBS/data/20220517_P185_laqueus_DiI_L-M27/analysis/indexing/channel_LUT.csv",
        "offset": 0,
        "hdf5_column": 4,
    },
    {
        "name": "L-A1",
        "channel_lut": "/bucket/ReiterU/DBS/data/20220613_P200_laqueus_DiI_L-A1/analysis/indexing/channel_LUT.csv",
        "offset": 0,
        "hdf5_column": 5,
    },
    {
        "name": "L-A2",
        "channel_lut": "/bucket/ReiterU/DBS/data/20220518_P186_laqueus_DiI_L-A2/analysis/indexing/channel_LUT.csv",
        "offset": 10,
        "hdf5_column": 6,
    },
    {
        "name": "L-A11",
        "channel_lut": "/bucket/ReiterU/DBS/data/20220614_P201_laqueus_DiI_L-A11/analysis/indexing/channel_LUT.csv",
        "offset": 0,
        "hdf5_column": 7,
    },
    {
        "name": "L-A29",
        "channel_lut": "/bucket/ReiterU/DBS/data/20220615_P202_laqueus_DiI_L-A29/analysis/indexing/channel_LUT.csv",
        "offset": 0,
        "hdf5_column": 8,
    }
]

In [6]:
hdf_info = [
    {
        "path": "/bucket/ReiterU/octopus/electrophysiology/asBoutOverChans.h5",
        "key": "maxL",
        "name": "AS_low",
        "description": "Envelop intensity (a.u.) in 0.1-10 Hz band during AS"
    },
    {
        "path": "/bucket/ReiterU/octopus/electrophysiology/asBoutOverChans.h5",
        "key": "maxH",
        "name": "AS_high",
        "description": "Envelop intensity (a.u.) in 20-40 Hz band during AS"
    },
    {
        "path": "/bucket/ReiterU/octopus/electrophysiology/wakeBoutOverChans.h5",
        "key": "maxL",
        "name": "wake_low",
        "description": "Envelop intensity (a.u.) in 0.1-10 Hz band during wake"
    },
    {
        "path": "/bucket/ReiterU/octopus/electrophysiology/wakeBoutOverChans.h5",
        "key": "maxH",
        "name": "wake_high",
        "description": "Envelop intensity (a.u.) in 20-40 Hz band during wake"
    },
    {
        "path": "/bucket/ReiterU/octopus/electrophysiology/spindleRate.h5",
        "key": "smoothedRate",
        "name": "spindle_rate",
        "description": "Spindle rate (events per min) during QS"
    },
    {
        "path": "/bucket/ReiterU/octopus/electrophysiology/bceBoutOverChans.h5",
        "key": "maxBCE",
        "name": "BCE-LFP_xcr",
        "description": "Correlation between BCE and LFP during QS"
    },
    {
        "path": "/bucket/ReiterU/octopus/electrophysiology/vleRate.h5",
        "key": "smoothedRate",
        "name": "VLE_rate",
        "description": "VLE rate"
    },
]

In [7]:
probes = merge_data_oct(data, hdf_info, lut)

In [8]:
# remove aBL channels, since the sample size is too low
probes.drop(probes[probes["Region_name"]=="aBL"].index, inplace=True)

In [9]:
display(probes)

,channel,X,Y,Z,X_org,Y_org,Z_org,Region,Region_name,Region_color,AS_low,AS_high,wake_low,wake_high,spindle_rate,BCE-LFP_xcr,VLE_rate,probe
0,0,188.530,182.238,151.016,1787.73,2235.010,1369.73,7,Subfr,"[251, 202, 0]",64.738417,13.477664,72.926777,8.887183,0.002035,1.994341,0.009922,L-J30
1,1,189.106,182.020,151.850,1790.54,2227.030,1374.33,6,Buc,"[244, 128, 26]",68.124028,13.950383,108.035811,9.323205,0.002358,2.057828,0.009922,L-J30
2,2,189.691,181.804,152.705,1793.35,2219.050,1378.92,6,Buc,"[244, 128, 26]",70.321409,14.071833,131.417497,9.602786,0.002367,2.068224,0.011709,L-J30
3,3,190.277,181.584,153.564,1796.16,2211.070,1383.52,6,Buc,"[244, 128, 26]",68.298372,13.935044,143.803293,9.909288,0.002367,2.149876,0.011709,L-J30
4,4,190.866,181.371,154.449,1798.97,2203.080,1388.11,6,Buc,"[244, 128, 26]",69.225287,14.013463,173.775585,9.867201,0.002619,2.183096,0.014387,L-J30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1963,182,242.226,151.987,424.261,1992.10,478.734,2870.82,1,VL,"[128, 174, 128]",113.846435,7.493584,115.169538,10.274815,0.001230,2.154674,0.006667,L-A29
1964,183,242.312,151.292,425.283,1991.75,469.050,2873.30,1,VL,"[128, 174, 128]",95.406197,7.262989,107.238174,9.933186,0.001230,2.075590,0.005833,L-A29
1965,184,242.384,150.592,426.303,1991.21,459.367,2875.72,1,VL,"[128, 174, 128]",93.702774,7.556474,129.645912,10.377255,0.001230,2.282939,0.005833,L-A29
1966,185,242.443,149.871,427.341,1990.49,449.684,2878.09,1,VL,"[128, 174, 128]",85.800312,7.260943,98.301750,9.732888,0.001230,2.059630,0.003333,L-A29


In [11]:
threeD.render_probes_on_surface(probes, "all", "spindle_rate", "/bucket/ReiterU/DBS/atlas/O_Laqueus_v0.2/",
                                vmin=0, vmax=0.075,
                                surface_vmin=0, surface_vmax=0.05,
                                surface_coloring_by="intensity", surface_colormap="Reds",
                                savename=os.path.join(tmpdir, "3D_spindlerate.png"))

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

...

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

RuntimeError: wrapped C/C++ object of type QAction has been deleted

### Save results in /bucket

In [12]:
resultdir = "/bucket/ReiterU/DBS/analysis/221020_octopus_spindle_figures"
if not os.path.exists(resultdir):
    subprocess.run(["ssh", "deigo", "mkdir", resultdir])

In [13]:
subprocess.run(["scp", "-r", tmpdir, f"deigo:{resultdir}"])

CompletedProcess(args=['scp', '-r', '/flash/ReiterU/tmp/221021_octopus_spindle_figures/3D', 'deigo:/bucket/ReiterU/DBS/analysis/221020_octopus_spindle_figures'], returncode=0)